In [1]:
import dill
from copy import deepcopy
import time
import random
import numpy as np
from sklearn.metrics import roc_curve, auc

import nltk

nltk.download("punkt")
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn

from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data import BucketIterator
from torchtext.data import Iterator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
RANDOM_SEED = 2020
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# DATA_PATH = "data/processed/"
DATA_PATH = "/content/"

## Dataset

In [3]:
import pandas as pd


train_df = []
valid_df = []
test_df = []
for data_name in ["sat", "cola"]:
    train_df += [pd.read_csv(f"{DATA_PATH}/{data_name}_train.tsv", sep="\t")]
    valid_df += [pd.read_csv(f"{DATA_PATH}/{data_name}_valid.tsv", sep="\t")]

train_df = pd.concat(train_df)
valid_df = pd.concat(valid_df)

train_df.to_csv("mix_train.tsv", sep="\t", index=False)
valid_df.to_csv("mix_valid.tsv", sep="\t", index=False)

In [4]:
TEXT = Field(
    sequential=True,
    use_vocab=True,
    tokenize=word_tokenize,
    lower=True,
    batch_first=True,
)
LABEL = Field(
    sequential=False,
    use_vocab=False,
    batch_first=True,
)


mix_train_data, mix_valid_data = TabularDataset.splits(
    path=DATA_PATH,
    train="mix_train.tsv",
    validation="mix_valid.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1,
)

TEXT.build_vocab(mix_train_data, min_freq=2)


mix_train_iterator, mix_valid_iterator = BucketIterator.splits(
    (mix_train_data, mix_valid_data),
    batch_size=32,
    device=None,
    sort=False,
)

sat_test_data = TabularDataset(
    path=f"{DATA_PATH}/sat_test.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1
)

sat_test_iterator = BucketIterator(
    sat_test_data,
    batch_size=8, 
    device=None,
    sort=False,
    shuffle=False
)

## LSTM Pooling Classifier

In [5]:
class LSTMPoolingClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super(LSTMPoolingClassifier, self).__init__()
        self.embed_layer = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=pad_idx)
        self.hidden_size = hidden_size
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        self.ih2h = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers,
                            bidirectional=True, batch_first=True, dropout=0.5)
        self.pool2o = nn.Linear(2 * hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.embed_layer(x)
        o, _ = self.ih2h(x)
        pool = nn.functional.max_pool1d(o.transpose(1, 2), x.shape[1])
        pool = pool.transpose(1, 2).squeeze()
        pool = self.dropout(pool)
        output = self.sigmoid(self.pool2o(pool))
        return output.squeeze()

In [6]:
def train(model: nn.Module,
          iterator: Iterator,
          optimizer: torch.optim.Optimizer,
          criterion: nn.Module,
          device: str):
    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):
        optimizer.zero_grad()

        text = batch.text
        if text.shape[0] > 1:
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)

            output = model(text).flatten()
            loss = criterion(output, label)
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: Iterator,
             criterion: nn.Module,
             device: str):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for _, batch in enumerate(iterator):
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def test(
    model: nn.Module,
    iterator: Iterator,
    device: str):

    with torch.no_grad():
        y_real = []
        y_pred = []
        model.eval()
        for batch in iterator:
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)

            output = model(text).flatten().cpu()

            y_real += [label]
            y_pred += [output]

        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)

    fpr, tpr, _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)

    return auroc

def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Pretrain with cola dataset

In [7]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20

lstm_classifier = LSTMPoolingClassifier(
    num_embeddings=len(TEXT.vocab),
    embedding_dim=100,
    hidden_size=200,
    num_layers=4,
    pad_idx=PAD_IDX,
)
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
_ = lstm_classifier.to(device)

optimizer = torch.optim.Adam(lstm_classifier.parameters())
bce_loss_fn = nn.BCELoss()

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(lstm_classifier, mix_train_iterator, optimizer, bce_loss_fn, device)
    valid_loss = evaluate(lstm_classifier, mix_valid_iterator, bce_loss_fn, device)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.5f}")
    print(f"\t Val. Loss: {valid_loss:.5f}")

test_auroc = test(lstm_classifier, sat_test_iterator, device)

print(f"| SAT Dataset Test AUROC: {test_auroc:.5f}")

Epoch: 01 | Time: 0m 5s
	Train Loss: 0.60662
	 Val. Loss: 0.65990
Epoch: 02 | Time: 0m 4s
	Train Loss: 0.59503
	 Val. Loss: 0.65051
Epoch: 03 | Time: 0m 4s
	Train Loss: 0.56600
	 Val. Loss: 0.68920
Epoch: 04 | Time: 0m 4s
	Train Loss: 0.52828
	 Val. Loss: 0.69242
Epoch: 05 | Time: 0m 4s
	Train Loss: 0.47584
	 Val. Loss: 0.76790
Epoch: 06 | Time: 0m 4s
	Train Loss: 0.41448
	 Val. Loss: 0.84083
Epoch: 07 | Time: 0m 4s
	Train Loss: 0.35734
	 Val. Loss: 0.92759
Epoch: 08 | Time: 0m 4s
	Train Loss: 0.29494
	 Val. Loss: 1.15169
Epoch: 09 | Time: 0m 4s
	Train Loss: 0.24994
	 Val. Loss: 1.18178
Epoch: 10 | Time: 0m 4s
	Train Loss: 0.20554
	 Val. Loss: 1.26133
Epoch: 11 | Time: 0m 4s
	Train Loss: 0.16845
	 Val. Loss: 1.53324
Epoch: 12 | Time: 0m 4s
	Train Loss: 0.14884
	 Val. Loss: 1.58437
Epoch: 13 | Time: 0m 4s
	Train Loss: 0.11677
	 Val. Loss: 1.63313
Epoch: 14 | Time: 0m 4s
	Train Loss: 0.11594
	 Val. Loss: 1.62229
Epoch: 15 | Time: 0m 4s
	Train Loss: 0.09337
	 Val. Loss: 1.72785
Epoch: 16 

In [8]:
_ = lstm_classifier.cpu()
mix_lstm_sat_test_auroc = test(lstm_classifier, sat_test_iterator, "cpu")

print(f"| SAT Dataset Test AUROC: {mix_lstm_sat_test_auroc:.5f}")

| SAT Dataset Test AUROC: 0.34615


In [9]:
with open("mix_data_advanced_model.dill", "wb") as f:
    model = {
        "TEXT": TEXT,
        "LABEL": LABEL,
        "classifier": lstm_classifier
    }
    dill.dump(model, f)